In [1]:
import numpy as np
import pandas as pd

In [2]:
# helper functions
def map_new_to_old_style(sequence):
    # there are 4 types of landmarks [face,pose,right_hand,left_hand]
    types = []# list where we'll store landmarks
    landmark_indexes = []
    for column in list(sequence.columns)[1:544]:
        # first column is frame which is not a landmark so counter starts from 1
        # why 544 - because we are given that there are now 1,629 spatial coordinate columns for the x, y and z coordinates for each of the 543 landmarks.
        parts = column.split("_")
        if len(parts) == 4:
            # for x_left_hand_1 - there will be 4 parts and for x_pose_1 there will be 3 only
            types.append(parts[1] + "_" + parts[2])
        else:
            types.append(parts[1])

        landmark_indexes.append(int(parts[-1]))

    data = {"frame": [], "type": [], "landmark_index": [], "x": [], "y": [], "z": []}

    for index, row in sequence.iterrows():
        data["frame"] += [int(row.frame)] * 543
        #       frame has values from 1,2,3....
        # 1. Firstly we are converting them into integers - 1
        # 2. Then we are making it a list - [1]
        # 3. Multiplying by 543 makes the replicates the same element(1 from our example) 543 times for all the landmarks present
        data["type"] += types
        data["landmark_index"] += landmark_indexes

        for _type, landmark_index in zip(types, landmark_indexes):
            data["x"].append(row[f"x_{_type}_{landmark_index}"])
            data["y"].append(row[f"y_{_type}_{landmark_index}"])
            data["z"].append(row[f"z_{_type}_{landmark_index}"])

    return pd.DataFrame.from_dict(data)


def assign_colors(row):
    if row == "face":
        return "red"
    elif "hand" in row:
        return "blue"
    else:
        return "green"


# specifies the plotting order
def assign_order(row):
    if row.type == "face":
        return row.landmark_index + 101
    elif row.type == "pose":
        return row.landmark_index + 30
    elif row.type == "left_hand":
        return row.landmark_index + 80
    else:
        return row.landmark_index

In [3]:
# A function to visualize the landmarks in 2d
def visualise2d_landmarks(parquet_df, title=""):
    # we first define a list of landmark connections, which specify which landmarks are connected by a line.

    # face landmarks are not connected by lines, you can also see that we have added 101 to face landmark indexs and in connections all values are below 100

    connections = [  
        [0, 1, 2, 3, 4,],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]

    parquet_df = map_new_to_old_style(parquet_df)
    frames = sorted(set(parquet_df.frame))
    first_frame = min(frames)
    parquet_df["color"] = parquet_df.type.apply(lambda row: assign_colors(row))
    parquet_df["plot_order"] = parquet_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = parquet_df[parquet_df.frame == first_frame].copy()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index("plot_order")

    frames_l = []
    for frame in frames:
        filtered_df = parquet_df[parquet_df.frame == frame].copy()
        filtered_df = filtered_df.sort_values(["plot_order"]).set_index("plot_order")
        traces = [
            go.Scatter(
                x=filtered_df["x"],
                y=filtered_df["y"],
                mode="markers",
                marker=dict(color=filtered_df.color, size=9),
            )
        ]

        for i, seg in enumerate(connections):
            trace = go.Scatter(
                x=filtered_df.loc[seg]["x"],
                y=filtered_df.loc[seg]["y"],
                mode="lines",
            )
            traces.append(trace)
        frame_data = go.Frame(data=traces, traces=[i for i in range(17)])
        frames_l.append(frame_data)

    traces = [
        go.Scatter(
            x=first_frame_df["x"],
            y=first_frame_df["y"],
            mode="markers",
            marker=dict(color=first_frame_df.color, size=9),
        )
    ]
    for i, seg in enumerate(connections):
        trace = go.Scatter(
            x=first_frame_df.loc[seg]["x"],
            y=first_frame_df.loc[seg]["y"],
            mode="lines",
            line=dict(color="black", width=2),
        )
        traces.append(trace)
    fig = go.Figure(data=traces, frames=frames_l)

    fig.update_layout(
        width=500,
        height=800,
        scene={
            "aspectmode": "data",
        },
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [
                            None,
                            {
                                "frame": {"duration": 100, "redraw": True},
                                "fromcurrent": True,
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "&#9654;",
                        "method": "animate",
                    },
                    {
                        "args": [
                            [None],
                            {
                                "frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "&#9612;&#9612;",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 100, "t": 100},
                "font": {"size": 20},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
    )
    camera = dict(up=dict(x=0, y=-1, z=0), eye=dict(x=0, y=0, z=2.5))
    fig.update_layout(title_text=title, title_x=0.5)
    fig.update_layout(scene_camera=camera, showlegend=False)
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
    )
    fig.update_yaxes(autorange="reversed")

    fig.show()


def get_phrase(df, file_id, sequence_id):
    return df[
        np.logical_and(df.file_id == file_id, df.sequence_id == sequence_id)
    ].phrase.iloc[0]


# Importing Libraries




In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objs as go
import json

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
supplement_data = pd.read_csv('/kaggle/input/asl-fingerspelling/supplemental_metadata.csv')
train_data = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')

In [6]:
train_data.shape

(67208, 5)

In [7]:
supplement_data.shape

(52958, 5)

# EDA

In [8]:
supplement_data.head()

,path,file_id,sequence_id,participant_id,phrase
0,supplemental_landmarks/33432165.parquet,33432165,1535467051,251,coming up with killer sound bites
1,supplemental_landmarks/33432165.parquet,33432165,1535499058,239,we better investigate this
2,supplemental_landmarks/33432165.parquet,33432165,1535530550,245,interesting observation was made
3,supplemental_landmarks/33432165.parquet,33432165,1535545499,38,victims deserve more redress
4,supplemental_landmarks/33432165.parquet,33432165,1535585216,254,knee bone is connected to the thigh bone


In [9]:
print("Number of rows and columns:", supplement_data.shape) 
print()
print('*************************************')
print()
print(supplement_data.info())
print()
print('*************************************')
print()
print("Number of Unique Values:\n",supplement_data.nunique())
print()
print('*************************************')
print()
print("Duplicate Values:\n",supplement_data.duplicated().sum())
print()
print('*************************************')
print()
print("Null Values:\n",supplement_data.isnull().sum())

Number of rows and columns: (52958, 5)

*************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52958 entries, 0 to 52957
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            52958 non-null  object
 1   file_id         52958 non-null  int64 
 2   sequence_id     52958 non-null  int64 
 3   participant_id  52958 non-null  int64 
 4   phrase          52958 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.0+ MB
None

*************************************

Number of Unique Values:
 path                 53
file_id              53
sequence_id       52958
participant_id       72
phrase              508
dtype: int64

*************************************

Duplicate Values:
 0

*************************************

Null Values:
 path              0
file_id           0
sequence_id       0
participant_id    0
phrase            0
dtype: int64




Observations :

    There are 52958 rows and 5 columns.
    There are no null values.
    There are no duplicate records.



In [10]:
supplement_data.describe()

,file_id,sequence_id,participant_id
count,5.295800e+04,5.295800e+04,52958.000000
mean,9.680392e+08,1.072800e+09,132.738661
std,5.779288e+08,6.165748e+08,81.745528
min,3.343216e+07,2.869900e+04,0.000000
25%,4.717666e+08,5.411308e+08,53.000000
50%,8.972877e+08,1.069840e+09,135.000000
75%,1.471342e+09,1.606032e+09,216.000000
max,2.100074e+09,2.147473e+09,254.000000



# For Train data


In [11]:
train_data.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [12]:
print("Number of rows and columns:", train_data.shape) 
print()
print('*************************************')
print()
print(train_data.info())
print()
print('*************************************')
print()
print("Number of Unique Values:\n",train_data.nunique())
print()
print('*************************************')
print()
print("Duplicate Values:\n",train_data.duplicated().sum())
print()
print('*************************************')
print()
print("Null Values:\n",train_data.isnull().sum())

Number of rows and columns: (67208, 5)

*************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67208 entries, 0 to 67207
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            67208 non-null  object
 1   file_id         67208 non-null  int64 
 2   sequence_id     67208 non-null  int64 
 3   participant_id  67208 non-null  int64 
 4   phrase          67208 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.6+ MB
None

*************************************

Number of Unique Values:
 path                 68
file_id              68
sequence_id       67208
participant_id       94
phrase            46478
dtype: int64

*************************************

Duplicate Values:
 0

*************************************

Null Values:
 path              0
file_id           0
sequence_id       0
participant_id    0
phrase            0
dtype: int64




Observations :

    There are 67287 rows and 5 columns.
    There are no null values.
    There are no duplicate records.




# Getting to know the length of each phrase.


In [13]:
supplement_data['phrase_char_len'] = supplement_data['phrase'].apply(len)
supplement_data.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_char_len
0,supplemental_landmarks/33432165.parquet,33432165,1535467051,251,coming up with killer sound bites,33
1,supplemental_landmarks/33432165.parquet,33432165,1535499058,239,we better investigate this,26
2,supplemental_landmarks/33432165.parquet,33432165,1535530550,245,interesting observation was made,32
3,supplemental_landmarks/33432165.parquet,33432165,1535545499,38,victims deserve more redress,28
4,supplemental_landmarks/33432165.parquet,33432165,1535585216,254,knee bone is connected to the thigh bone,40


In [14]:
fig = px.histogram(supplement_data,x='phrase_char_len',nbins=35,color_discrete_sequence = px.colors.qualitative.Set2, title="Virus affected different age groups")
fig.show()



Observations :

    It seems that the number of records are having maximum count for the phrase length between 25-35.



In [15]:
total_phrases = supplement_data['phrase'].value_counts()

data_phrases = pd.DataFrame({'phrases': total_phrases.index, 'phrase count': total_phrases.values})

In [16]:
data_phrases.head()

,phrases,phrase count
0,why do you ask silly questions,117
1,find a nearby parking spot,117
2,apartments are too expensive,116
3,what to do when the oil runs dry,115
4,that agreement is rife with problems,115


In [17]:
data_phrases.shape

(508, 2)

# Visualizing the phrases count in a 'phrase' column.

In [18]:
fig = px.bar(data_phrases.iloc[:10,:],x='phrase count', y='phrases', height=700, width= 1200, color='phrases',color_discrete_sequence =px.colors.qualitative.G10, title='Top 10 Most frequently used phrases')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


# Least used phrases.


In [19]:
fig = px.bar(data_phrases.iloc[-10:,:],x='phrase count', y='phrases', height=700, width= 1200, color='phrases',color_discrete_sequence= px.colors.sequential.Plasma, title='Top 10 Least used phrases')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

# Exploring landmarks file
# Most used phrase

In [20]:
# Creating subset of dataset where phrase is "why do you ask silly questions".

top_phrase = supplement_data[supplement_data["phrase"]=="why do you ask silly questions"]['path'].values[0]
top_phrase

'supplemental_landmarks/86446671.parquet'

# Least used phrase

In [21]:
# Creating subset of dataset where phrase is "my favorite place is to visit".

bottom_phrase = supplement_data[supplement_data["phrase"]=="my favorite place is to visit"]['path'].values[0]
bottom_phrase

'supplemental_landmarks/1471341722.parquet'

# Exploring data related to top_phrase further

In [22]:
landmark_df = pd.read_parquet('/kaggle/input/asl-fingerspelling/'+ top_phrase)
landmark_df.head()

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
731203706,0,0.703698,0.688928,0.695702,0.679743,0.687841,0.689391,0.695388,0.609348,0.696638,...,-0.089671,-0.080566,-0.043455,-0.095483,-0.084067,-0.060941,-0.052851,-0.087094,-0.075263,-0.054956
731203706,1,0.715093,0.699307,0.705917,0.691370,0.698360,0.700104,0.706936,0.622183,0.708591,...,-0.077541,-0.070044,-0.025600,-0.081992,-0.077576,-0.053929,-0.039335,-0.071657,-0.060017,-0.036933
731203706,2,0.717082,0.707563,0.713474,0.698543,0.706523,0.707614,0.712466,0.623232,0.712809,...,-0.092923,-0.090480,-0.022318,-0.080716,-0.084622,-0.066322,-0.034716,-0.062594,-0.060636,-0.047767
731203706,3,0.719097,0.711470,0.717952,0.703294,0.710563,0.712090,0.718152,0.626309,0.719156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731203706,4,0.726000,0.713415,0.720095,0.706134,0.712599,0.714675,0.722513,0.633579,0.724624,...,-0.023693,-0.027201,-0.007065,-0.016289,-0.007165,0.000971,-0.010739,-0.015896,-0.005026,0.005931


In [23]:
landmark_df.shape

(171404, 1630)

In [24]:
landmark_df=landmark_df.reset_index(inplace=False)
landmark_df.head()

,sequence_id,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,731203706,0,0.703698,0.688928,0.695702,0.679743,0.687841,0.689391,0.695388,0.609348,...,-0.089671,-0.080566,-0.043455,-0.095483,-0.084067,-0.060941,-0.052851,-0.087094,-0.075263,-0.054956
1,731203706,1,0.715093,0.699307,0.705917,0.691370,0.698360,0.700104,0.706936,0.622183,...,-0.077541,-0.070044,-0.025600,-0.081992,-0.077576,-0.053929,-0.039335,-0.071657,-0.060017,-0.036933
2,731203706,2,0.717082,0.707563,0.713474,0.698543,0.706523,0.707614,0.712466,0.623232,...,-0.092923,-0.090480,-0.022318,-0.080716,-0.084622,-0.066322,-0.034716,-0.062594,-0.060636,-0.047767
3,731203706,3,0.719097,0.711470,0.717952,0.703294,0.710563,0.712090,0.718152,0.626309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,731203706,4,0.726000,0.713415,0.720095,0.706134,0.712599,0.714675,0.722513,0.633579,...,-0.023693,-0.027201,-0.007065,-0.016289,-0.007165,0.000971,-0.010739,-0.015896,-0.005026,0.005931


In [25]:
landmark_df.shape, landmark_df["sequence_id"].nunique()

((171404, 1631), 1000)


    There are 171404 rows and 1631 columns.
    There are 1000 unique sequence_id values.


# Reading Json Data

In [26]:
file_path = '/kaggle/input/asl-fingerspelling/character_to_prediction_index.json'

# Open the JSON file and load its contents
with open(file_path, 'r') as file:
    data = json.load(file)

print(data)

{' ': 0, '!': 1, '#': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '=': 27, '?': 28, '@': 29, '[': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57, '~': 58}




# Observations :

    It seems there is a relation assigned between character and value.
    We need to put it into more simple format.



In [27]:
char=[]
value=[]

for i,j in data.items():
    char.append(i)
    value.append(j)

In [28]:
char_to_pred_index=pd.DataFrame({"char":char,"value":value})
char_to_pred_index.head(20)

,char,value
0,,0
1,!,1
2,#,2
3,$,3
4,%,4
5,&,5
6,',6
7,(,7
8,),8
9,*,9


In [29]:
char_to_pred_index.shape

(59, 2)

# Visualizing the Sequence



# Generating a random integer between 0 and 53 (unique_file_ids)


In [30]:

unique_file_ids = len(np.unique(supplement_data['file_id']))
unique_file_ids

53

In [31]:
random_id = np.random.randint(0, unique_file_ids)

# Getting the random file_id 
random_file_id = np.unique(supplement_data['file_id'])[random_id]

# Getting all different sequences in random file

In [32]:
signs = supplement_data[supplement_data['file_id'] == random_file_id]
signs.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_char_len
51958,supplemental_landmarks/2100073719.parquet,2100073719,1051666924,89,please provide your date of birth,33
51959,supplemental_landmarks/2100073719.parquet,2100073719,1051689435,53,we dine out on the weekends,27
51960,supplemental_landmarks/2100073719.parquet,2100073719,1051814272,219,pay off a mortgage for a house,30
51961,supplemental_landmarks/2100073719.parquet,2100073719,1051820147,81,take it to the recycling depot,30
51962,supplemental_landmarks/2100073719.parquet,2100073719,1051842633,112,question that must be answered,30


In [33]:
signs.shape

(1000, 6)

In [34]:
print('Unique indexes : ',len(np.unique(signs.index)))

Unique indexes :  1000



# Getting a random Sequence id


In [35]:
random_squence_id = signs.sample()['sequence_id'].item()


# Locating the simultaneous parquet record using the random_file_id generated

In [36]:
path_to_sign = f"/kaggle/input/asl-fingerspelling/supplemental_landmarks/{random_file_id}.parquet"
parquet = pd.read_parquet(path_to_sign)

In [37]:
sequence = parquet[parquet.index == random_squence_id]
sequence

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1075257130,0,0.677427,0.663305,0.673244,0.666178,0.664462,0.670979,0.690250,0.621497,0.697843,...,-0.160852,-0.160095,-0.054142,-0.125753,-0.129293,-0.111550,-0.064253,-0.106454,-0.118458,-0.117280
1075257130,1,0.674935,0.661241,0.671200,0.664617,0.662517,0.669226,0.688968,0.619976,0.696814,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075257130,2,0.673458,0.659308,0.669475,0.663095,0.660583,0.667445,0.687732,0.620225,0.695844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075257130,3,0.674120,0.660857,0.670861,0.664688,0.662227,0.669094,0.689123,0.620098,0.697208,...,-0.129458,-0.129236,-0.048381,-0.098838,-0.099947,-0.085605,-0.066488,-0.087563,-0.089765,-0.085461
1075257130,4,0.672726,0.657355,0.667678,0.662679,0.659104,0.666556,0.688004,0.620016,0.696779,...,-0.126651,-0.125158,-0.044814,-0.100081,-0.106107,-0.093956,-0.059144,-0.085788,-0.092934,-0.091532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075257130,117,0.746855,0.739132,0.745593,0.718499,0.735934,0.733304,0.728170,0.629358,0.723337,...,-0.218976,-0.232670,-0.113160,-0.200550,-0.211655,-0.197272,-0.127905,-0.189913,-0.194746,-0.184389
1075257130,118,0.749210,0.740144,0.746363,0.719508,0.736998,0.734357,0.729071,0.629437,0.724204,...,-0.184880,-0.191665,-0.095120,-0.174013,-0.182330,-0.164789,-0.112391,-0.164368,-0.165836,-0.152194
1075257130,119,0.747986,0.737716,0.744482,0.718311,0.734720,0.732563,0.728632,0.630895,0.724353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Visualizing the Fingerspelling Animation

In [38]:
sequence_phrase = get_phrase(supplement_data, random_file_id, random_squence_id)
visualise2d_landmarks(sequence, f"Phrase: {sequence_phrase}")